In [631]:
import importlib
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.dates as md
import plotly.graph_objects as go

from dotenv import load_dotenv

In [632]:
# Makes sure modules are available to the notebook that acts as an entrypoint

load_dotenv()

parent_dir = os.path.abspath('..')

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [633]:
# Manually reloading python module such that jupyter reflects changes without kernel restart

import plutus.code.calc.dm as dm
import plutus.code.calc.dmv2 as dmv2
import plutus.code.calc.atr as atr

importlib.reload(dmv2);
importlib.reload(atr)

<module 'plutus.code.calc.atr' from '/Users/stanislavkonnov/dev/success-factory/stock-insiders/plutus/plutus/code/calc/atr.py'>

In [634]:
import plutus.code.api_connector as ac

importlib.reload(ac);

In [635]:
ticker = 'TSLA'
start_date = '2023-05-24-00:00'
end_date = '2023-05-25-00:00'

api_connector = ac.ApiConnector()
api_connector.ticker = ticker
api_connector.start_date = start_date
api_connector.end_date = end_date

df = api_connector.request_daily_prices()

df

Price for TSLA are available on disk. Reading


datetime   close    high     low    open  volume
0   2023-05-24 08:00:00  184.04  184.69  184.04  184.43    6820
1   2023-05-24 08:01:00  184.20  184.26  184.10  184.10    7256
2   2023-05-24 08:02:00  184.30  184.30  184.16  184.21    7687
3   2023-05-24 08:03:00  184.30  184.39  184.29  184.39    1806
4   2023-05-24 08:04:00  184.25  184.30  184.25  184.26    3717
..                  ...     ...     ...     ...     ...     ...
933 2023-05-24 23:55:00  185.10  185.10  185.01  185.05    1803
934 2023-05-24 23:56:00  185.00  185.05  185.00  185.01    4215
935 2023-05-24 23:57:00  185.00  185.10  184.96  184.99    3754
936 2023-05-24 23:58:00  185.15  185.15  185.08  185.09    4831
937 2023-05-24 23:59:00  185.25  185.25  185.09  185.16   15423

[938 rows x 6 columns]

In [636]:
# dm_calculator = dm.DistributionMomentsCalculator()

# dm_calculator.dataframe = df

# moving_skewness = dm_calculator.calculate_moving_dist_moments()

window_size = 180

dm_c = dmv2.DistributionMomentsCalculator()
dm_c.dataframe = df
dm_c.window_size = window_size
dm_c.calculate_moving_dist_moments(mean=False, std=False, skew=True, kurt=True)

at_c = atr.AverageTrueRangeCalculator()
at_c.dataframe = df
at_c.window_size = window_size
at_c.calculate_average_true_range()

df

datetime   close    high     low    open  volume      skew  \
0   2023-05-24 08:00:00  184.04  184.69  184.04  184.43    6820       NaN   
1   2023-05-24 08:01:00  184.20  184.26  184.10  184.10    7256       NaN   
2   2023-05-24 08:02:00  184.30  184.30  184.16  184.21    7687       NaN   
3   2023-05-24 08:03:00  184.30  184.39  184.29  184.39    1806       NaN   
4   2023-05-24 08:04:00  184.25  184.30  184.25  184.26    3717       NaN   
..                  ...     ...     ...     ...     ...     ...       ...   
933 2023-05-24 23:55:00  185.10  185.10  185.01  185.05    1803  0.258217   
934 2023-05-24 23:56:00  185.00  185.05  185.00  185.01    4215  0.267875   
935 2023-05-24 23:57:00  185.00  185.10  184.96  184.99    3754  0.268196   
936 2023-05-24 23:58:00  185.15  185.15  185.08  185.09    4831  0.267207   
937 2023-05-24 23:59:00  185.25  185.25  185.09  185.16   15423  0.264983   

         kurt       atr  
0         NaN       NaN  
1         NaN       NaN  
2         NaN       NaN  
3         NaN       NaN  
4         NaN       NaN  
..        ...       ...  
933 -0.793841  0.201909  
934 -0.785614  0.201343  
935 -0.781726  0.201002  
936 -0.776310  0.200719  
937 -0.769889  0.200493  

[938 rows x 9 columns]

In [637]:
prime_value = 'close'
trace_value = 'skew'

x=df.index.values

y1=df[prime_value].values
y2=df[trace_value].values

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis='y2')

# Create the layout with two y-axes
layout = go.Layout(
    title=f"{ticker} {df['datetime'].values[0]} - {df['datetime'].values[-1]}",
    yaxis=dict(),
    yaxis2=dict( overlaying='y', side='right'),
    height=650
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)
fig.update_yaxes(showspikes=True, spikemode="across", spikecolor="black", spikethickness=0.5)